# Lab 5 – Hyperparameter Tuning and Optimization
Run a small sweep over LR, LoRA rank, and gradient accumulation. Pick best by validation perplexity.

## Step 0. Stable installs

In [1]:
%pip install -q --force-reinstall numpy==2.0.2 pandas==2.2.2 pyarrow==17.0.0
%pip install -q datasets>=3.0.0 transformers>=4.41.0 peft>=0.11.0 accelerate>=0.29.0 sentencepiece>=0.1.99 tqdm>=4.66.0 bitsandbytes
print('If imports fail, use Runtime → Restart runtime and re-run this cell.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 26.3 MB/s eta 0:00:00
If imports fail, use Runtime → Restart runtime and re-run this cell.


## Step 1. Auto-detect dataset in Drive

In [2]:
# Step 1: Load dataset saved from Lab 3
from google.colab import drive
from pathlib import Path
from datasets import load_from_disk

drive.mount('/content/drive')

# Consistent base directory for all labs
BASE_DIR = Path("/content/drive/MyDrive/slm-labs")
BASE_DIR.mkdir(parents=True, exist_ok=True)

# Dataset directory from Lab 3
DATA_DIR = BASE_DIR / "lab3_tokenized"

# Sanity checks
assert DATA_DIR.exists(), f"Dataset not found at {DATA_DIR}. Run Lab 3 first."

# Load dataset
dataset = load_from_disk(DATA_DIR)
print("Loaded dataset from:", DATA_DIR)
print("Splits:", list(dataset.keys()))
for split, dset in dataset.items():
    print(f"{split}: {len(dset)} rows, columns = {dset.column_names}")


Mounted at /content/drive
Loaded dataset from: /content/drive/MyDrive/slm-labs/lab3_tokenized
Splits: ['train']
train: 121736 rows, columns = ['input_ids', 'attention_mask', 'labels']


## Step 2. Load dataset

In [3]:
# Step 2: Load dataset (works only if Step 1 verified a proper folder)
from datasets import load_from_disk
ds = load_from_disk(str(DATA_DIR))
print(ds)
val = ds.get("validation") or ds.get("test")
if val is None:
    # fall back to a small slice of train for quick checks
    val = ds["train"].select(range(min(200, len(ds["train"]))))
print("Validation samples:", len(val))

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 121736
    })
})
Validation samples: 200


## Step 3. Load base model (4-bit if possible)

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
BASE_MODEL='HuggingFaceH4/zephyr-7b-beta'
kw={}
if torch.cuda.is_available():
    try:
        kw=dict(device_map='auto', quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type='nf4', bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True), torch_dtype=torch.float16)
    except Exception:
        kw=dict(torch_dtype=torch.float16)
else:
    kw=dict(torch_dtype=torch.float32)
Tok=AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
Model=AutoModelForCausalLM.from_pretrained(BASE_MODEL, **kw)
if Tok.pad_token is None: Tok.pad_token=Tok.eos_token

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Step 4. Attach LoRA

In [5]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
TARGETS=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]
def attach_lora(m, r=16, alpha=32, drop=0.05):
    m = prepare_model_for_kbit_training(m)
    cfg = LoraConfig(r=r, lora_alpha=alpha, lora_dropout=drop, target_modules=TARGETS, bias='none', task_type='CAUSAL_LM')
    pm = get_peft_model(m, cfg)
    pm.print_trainable_parameters()
    return pm

## Step 5. Train short and evaluate

In [6]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import math, time, pandas as pd

def run(h):
    m = attach_lora(Model, r=h['r'], alpha=h['alpha'], drop=h['drop'])
    coll = DataCollatorForLanguageModeling(tokenizer=Tok, mlm=False)
    args = TrainingArguments(output_dir=f"./out_{int(time.time())}", per_device_train_batch_size=h['bs'], gradient_accumulation_steps=h['ga'], learning_rate=h['lr'], warmup_steps=10, max_steps=h['steps'], logging_steps=10, save_strategy='no', fp16=torch.cuda.is_available(), report_to=[])
    trainer = Trainer(model=m, args=args, train_dataset=ds['train'], eval_dataset=val, data_collator=coll)
    trainer.train()
    ev = trainer.evaluate() if val else {}
    loss = ev.get('eval_loss', None)
    ppl = math.exp(loss) if loss else None
    return {'loss': loss, 'ppl': ppl}, m

search=[{'lr':2e-4,'r':16,'alpha':32,'drop':0.05,'bs':2,'ga':4,'steps':100}]
recs=[]; BEST=None; BEST_MODEL=None
for h in search:
    print('Trial',h)
    mtr, m=run(h)
    row={**h, **mtr}; recs.append(row)
    if BEST is None or (mtr['ppl'] and mtr['ppl']<BEST['ppl']):
        BEST=row; BEST_MODEL=m
DF=pd.DataFrame(recs)
display(DF)
print('Best:',BEST)

Trial {'lr': 0.0002, 'r': 16, 'alpha': 32, 'drop': 0.05, 'bs': 2, 'ga': 4, 'steps': 100}
trainable params: 41,943,040 || all params: 7,283,675,136 || trainable%: 0.5758


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.680100
20,1.631500
30,1.635800
40,1.629300
50,1.607500
60,1.644000
70,1.613600
80,1.602000
90,1.576800
100,1.605600


,lr,r,alpha,drop,bs,ga,steps,loss,ppl
0,0.0002,16,32,0.05,2,4,100,1.661893,5.269274


Best: {'lr': 0.0002, 'r': 16, 'alpha': 32, 'drop': 0.05, 'bs': 2, 'ga': 4, 'steps': 100, 'loss': 1.6618926525115967, 'ppl': 5.269274318048369}


## Step 6. Save results to Drive

In [7]:
from pathlib import Path
RES = Path('/content/drive/MyDrive/slm-labs/lab5_results'); RES.mkdir(parents=True, exist_ok=True)
DF.to_csv(RES/'trials.csv', index=False)
if BEST_MODEL is not None:
    tag=f"r{BEST['r']}_lr{BEST['lr']}_ga{BEST['ga']}"; sd=RES/f"best_{tag}"; sd.mkdir(parents=True, exist_ok=True)
    BEST_MODEL.save_pretrained(sd); Tok.save_pretrained(sd)
    print('Saved best adapters to', sd)
else:
    print('No best model to save')

Saved best adapters to /content/drive/MyDrive/slm-labs/lab5_results/best_r16_lr0.0002_ga4
